In [1]:
# ✅ Resea: Virtual Research Agent
# Uses Hugging Face + SerpAPI, generates .docx/.pdf reports with web interface

!pip uninstall -y serpapi -q
!pip install -q google-search-results transformers newspaper3k wikipedia python-docx lxml[html_clean] reportlab gradio

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 35.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# === Imports ===
import os
import re
import wikipedia
import gradio as gr
from urllib.parse import urlparse
from newspaper import Article
from serpapi import GoogleSearch
from transformers import pipeline
from docx import Document
from reportlab.pdfgen import canvas


In [3]:
# === SerpAPI Key ===
SERPAPI_KEY = "445e154e5df8fa8655d9031c56e672cbdfa5d37cb74d6d021ca54d3fb4000a76"  # Update for your API
os.environ['SERPAPI_API_KEY'] = SERPAPI_KEY

In [4]:
print('Set up for virtual research agent complete.')

Set up for virtual research agent complete.